In [4]:
import numpy as np
import tensorflow as tf
import gymnasium as gym 

In [2]:
%pip uninstall -y gym


Found existing installation: gym 0.26.2
Uninstalling gym-0.26.2:
  Successfully uninstalled gym-0.26.2
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install gymnasium


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/951.1 kB ? eta -:--:--
   ----------- ---------------------------- 262.1/951.1 kB ? eta -:--:--
   ---------------------------------------- 951.1/951.1 kB 5.3 MB/s  0:00:00

   -------------------- ------------------- 1/2 [gymnasium]
   -------------------- ------------------- 1/2 [gymnasium]
   -------------------- ------------------- 1/2 [gymnasium]
   -------------------- ------------------- 1/2 [gymnasium]
   -------------------- ------------------- 1/2 [gymnasium]
   -------------------- ------------------- 1/2 [gymnasium]
   -------------------- ------------------- 1/2 [gymnasium]
   -------------------- ------------------- 1/2 [gymnasium]
   -------------------- ------------------- 1/2 [gymnasium]
   -------------------- ------------------- 1/2 [gymnasium]
   -------------------- ------------------- 1/2 [gymnasium]
   -------------------- ----------------

In [5]:
%pip install pygame

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
    --------------------------------------- 0.3/10.6 MB ? eta -:--:--
   -------- ------------------------------- 2.4/10.6 MB 11.0 MB/s eta 0:00:01
   --------- ------------------------------ 2.6/10.6 MB 6.0 MB/s eta 0:00:02
   ------------- -------------------------- 3.7/10.6 MB 5.4 MB/s eta 0:00:02
   ------------------- -------------------- 5.2/10.6 MB 5.8 MB/s eta 0:00:01
   ---------------------- ----------------- 6.0/10.6 MB 5.5 MB/s eta 0:00:01
   ------------------------- -------------- 6.8/10.6 MB 5.3 MB/s eta 0:00:01
   ---------------------------- ----------- 7.6/10.6 MB 5.1 MB/s eta 0:00:01
   ------------------------------- -------- 8.4/10.6 MB 5.0 MB/s eta 0:00:01
   ----------------------------------- ---- 9.4/10.6 MB 4.9 MB/s eta 0:00:01
   -------------------------------------- - 10.2/10.6 MB 4.8 MB/s eta 0:00:01
   -------

In [1]:
import gymnasium as gym

env = gym.make("CartPole-v1", render_mode="human")
observation, info = env.reset()
for _ in range(1000):
    action = env.action_space.sample()
    observation, reward, terminated, truncated, info = env.step(action)
    if terminated or truncated:
        observation, info = env.reset()
env.close()


C:\Users\Shuvo\AppData\Roaming\Python\Python313\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


In [6]:
%pip install "gymnasium[classic-control]"


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
# Create the CartPole Environment
env = gym.make('CartPole-v1')

In [6]:
# Define the actor and critic networks
actor = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(env.action_space.n, activation='softmax')
])

critic = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [7]:
# Define optimizer and loss functions
actor_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
critic_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

In [ ]:


num_episodes = 1000
gamma = 0.99

for episode in range(num_episodes):
    state, info = env.reset()  # Gymnasium returns (obs, info)
    episode_reward = 0

    with tf.GradientTape(persistent=True) as tape:
        for t in range(1, 10000):  # Limit time steps
            # Choose an action using the actor
            action_probs = actor(np.array([state]))
            action = np.random.choice(env.action_space.n, p=action_probs.numpy()[0])

            # Take the chosen action and observe next state and reward
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated  # Gymnasium uses both

            # Compute the advantage
            state_value = critic(np.array([state]))[0, 0]
            next_state_value = critic(np.array([next_state]))[0, 0]
            advantage = reward + gamma * next_state_value - state_value

            # Compute actor and critic losses
            actor_loss = -tf.math.log(action_probs[0, action]) * advantage
            critic_loss = tf.square(advantage)

            episode_reward += reward

            # Update actor and critic
            actor_gradients = tape.gradient(actor_loss, actor.trainable_variables)
            critic_gradients = tape.gradient(critic_loss, critic.trainable_variables)
            actor_optimizer.apply_gradients(zip(actor_gradients, actor.trainable_variables))
            critic_optimizer.apply_gradients(zip(critic_gradients, critic.trainable_variables))

            state = next_state

            if done:
                break

    if episode % 10 == 0:
        print(f'Episode {episode}, Reward: {episode_reward}')

env.close()


Episode 0, Reward: 20.0
Episode 10, Reward: 48.0
Episode 20, Reward: 42.0
Episode 30, Reward: 21.0
Episode 40, Reward: 36.0
Episode 50, Reward: 42.0
Episode 60, Reward: 186.0
Episode 70, Reward: 37.0
